In [3]:
import tensorflow as tf
import keras

Using TensorFlow backend.


# MNIST data

In [4]:
# Import the dataset
from keras.datasets import mnist

# Import various components for model-building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend.
from keras import backend as K

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 10s 1us/step


In [7]:
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

## Multilayer Perceptron

In [9]:
# Change shape.
# Note that the images are 28x28 pixels, so in reshaping to arrays, you want
# 60,000 arrays of length 784 (one for each image).
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to `float32` for type consistency.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values so that they run from `0` to `1` instead of from `0` to `255` (256 values of pixels).
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
# So instead of one column with 10 values, create 10 binary columns.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


In [10]:
# Start with a simple sequential model.
model = Sequential()

# Add dense layers to create a fully connected MLP.
# Note that you specify an input shape for the first layer, but only the first layer.
# `relu` is the activation function used.
model.add(Dense(64, activation='relu', input_shape=(784,)))

# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))

# End with a number of units equal to the number of classes that you have for your outcome.
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


epochs = iterations of the model

batch size = number of samples to use in each step of improving the model

Batch size affects speed and negatively impacts accuracy

aka learning in bigger steps will affect what your model learns.

In [11]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 24us/step - loss: 0.4263 - accuracy: 0.8748 - val_loss: 0.1992 - val_accuracy: 0.9402
Epoch 2/10
60000/60000 [==============================] - 1s 20us/step - loss: 0.2060 - accuracy: 0.9390 - val_loss: 0.1402 - val_accuracy: 0.9583
Epoch 3/10
60000/60000 [==============================] - 1s 21us/step - loss: 0.1572 - accuracy: 0.9519 - val_loss: 0.1125 - val_accuracy: 0.9655
Epoch 4/10
60000/60000 [==============================] - 1s 20us/step - loss: 0.1313 - accuracy: 0.9601 - val_loss: 0.1097 - val_accuracy: 0.9668
Epoch 5/10
60000/60000 [==============================] - 1s 20us/step - loss: 0.1160 - accuracy: 0.9648 - val_loss: 0.0966 - val_accuracy: 0.9707
Epoch 6/10
60000/60000 [==============================] - 1s 20us/step - loss: 0.1045 - accuracy: 0.9679 - val_loss: 0.0995 - val_accuracy: 0.9708
Epoch 7/10
60000/60000 [==============================] - 1s 20us/s

## Convolutional neural networks

In [12]:
# Input image dimensions from your data
img_rows, img_cols = 28, 28
num_classes = 10

# The data, shuffled and split between training and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the model
model = Sequential()
# First convolutional layer (note the specification of shape)
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 63s 1ms/step - loss: 0.2571 - accuracy: 0.9209 - val_loss: 0.0579 - val_accuracy: 0.9805
Epoch 2/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0892 - accuracy: 0.9732 - val_loss: 0.0438 - val_accuracy: 0.9860
Epoch 3/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0675 - accuracy: 0.9803 - val_loss: 0.0339 - val_accuracy: 0.9890
Epoch 4/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0563 - accuracy: 0.9833 - val_loss: 0.0336 - val_accuracy: 0.9889
Epoch 5/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0498 - accuracy: 0.9853 - val_loss: 0.0289 - val_accuracy: 0.9898
Epoch 6/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0418 - accuracy: 0.9869 - val_loss: 0.0287 - val_accuracy: 0

## Hierarchical recurrent neural networks

In [13]:
# Training parameters
batch_size = 64
num_classes = 10
epochs = 3

# Embedding dimensions
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for hierarchical RNN
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using the `TimeDistributed` wrapper
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 106s 2ms/step - loss: 0.9106 - accuracy: 0.6965 - val_loss: 0.4843 - val_accuracy: 0.8421
Epoch 2/3
60000/60000 [==============================] - 96s 2ms/step - loss: 0.3841 - accuracy: 0.8771 - val_loss: 0.2538 - val_accuracy: 0.9248
Epoch 3/3
60000/60000 [==============================] - 96s 2ms/step - loss: 0.2428 - accuracy: 0.9245 - val_loss: 0.1960 - val_accuracy: 0.9408
Test loss: 0.19604802551865577
Test accuracy: 0.9408000111579895


In [24]:
# The data, shuffled and split between training and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Change shape.
# Note that the images are 28x28 pixels, so in reshaping to arrays, you want
# 60,000 arrays of length 784 (one for each image).
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to `float32` for type consistency.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values so that they run from `0` to `1` instead of from `0` to `255` (256 values of pixels).
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
# So instead of one column with 10 values, create 10 binary columns.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Start with a simple sequential model.
model = Sequential()

# Add dense layers to create a fully connected MLP.
# Note that you specify an input shape for the first layer, but only the first layer.
# `relu` is the activation function used.
model.add(Dense(64, activation='relu', input_shape=(784,)))

# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))

# End with a number of units equal to the number of classes that you have for your outcome.
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

60000 train samples
10000 test samples
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 64)                50240     
_________________________________________________________________
dropout_29 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_30 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_43 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_31 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_44 (Dens